In [ ]:
import os

import numpy
import pandas
import seaborn
import xarray

from src import knn

In [ ]:
seaborn.set()
figsize = (12, 5)

In [ ]:
data = pandas.read_csv('bootstrapped_dates.csv', index_col=0)

In [ ]:
future_ds_path = os.path.join(knn.DATA_STORE_PATH, 'GCMs/Amazon__pr_day_CanESM5_ssp126_r1i1p1f1_gn_20150101-21001231.nc')

In [ ]:
observed_location = os.path.join(knn.DATA_STORE_PATH, 'OBSERVATIONS/LLdM_AOI2/catalogo.csv')
locations = pandas.read_csv(observed_location)
lon, lat = locations[locations['CODIGO_CAT'] == 'Beni01'][['longitud', 'latitud']].values[0]

In [ ]:
gcm = xarray.open_dataset(future_ds_path)
gcm = knn.shift_longitude_from_360(gcm)
gcm = gcm.sel(lon=lon, lat=lat, method='nearest')

In [ ]:
start_date = data.index.min()
end_date = data.index.max()

In [ ]:
gcm = gcm.sel(time=slice(start_date, end_date))

In [ ]:
gcm_precip = gcm.pr.to_numpy() * (60 * 60 * 24)
data['gcm_precip'] = gcm_precip

In [ ]:
data[['historic_precip', 'gcm_precip']].plot(
    # ylim=(0, 1000),
    figsize=figsize)